# H+, or how to build a perfect human.


In [18]:
import ftplib
import pandas as pd
from Bio import Entrez

In [ ]:
server = 'ftp.ncbi.nlm.nih.gov'
collection = '/pub/clinvar/vcf_GRCh37/'
file = 'clinvar.vcf.gz'
local_filepath = 'clinvar.vcf.gz'

ftp = ftplib.FTP(server)
ftp.login()
ftp.cwd(collection)
ftp.retrbinary("RETR {}".format(file), open(local_filepath, 'wb').write)
ftp.retrbinary("RETR {}.tbi".format(file), open(local_filepath + '.tbi', 'wb').write)
ftp.quit()

In [77]:
!ls -lah

total 295352
drwxr-xr-x  19 b2w  staff   608B 31 мар 14:23 .
drwxr-xr-x  40 b2w  staff   1,3K 30 мар 15:24 ..
-rw-r--r--@  1 b2w  staff   6,0K 30 мар 17:10 .DS_Store
drwxr-xr-x   3 b2w  staff    96B 30 мар 16:53 .ipynb_checkpoints
-rw-r--r--@  1 b2w  staff    23M 30 мар 15:25 23&me.txt
-rw-r--r--@  1 b2w  staff    24M 30 мар 16:49 clinvar.vcf.gz
-rw-r--r--   1 b2w  staff   277K 30 мар 16:50 clinvar.vcf.gz.tbi
-rw-r--r--@  1 b2w  staff    26K 30 мар 16:57 intersected.vcf
-rw-r--r--   1 b2w  staff    20K 31 мар 14:22 lab06_hplus.ipynb
-rw-r--r--   1 b2w  staff    37M 30 мар 15:33 out.vcf
-rw-r--r--@  1 b2w  staff   3,5M 31 мар 14:23 prof.eff.vcf
-rw-r--r--@  1 b2w  staff   1,3M 31 мар 14:21 professor_intersected.vcf
-rw-r--r--   1 b2w  staff   2,3K 31 мар 14:21 professor_out.hh
-rw-r--r--   1 b2w  staff   1,4K 31 мар 14:21 professor_out.log
-rw-r--r--@  1 b2w  staff    23M 31 мар 14:21 professor_out.vcf
-rw-rw-r--@  1 b2w  staff    15M 31 мар 14:13 professor_snp.txt
-rw-r--r--@  1 b2w  s

In [73]:
with open('professor_snp.txt', 'r') as f:
    for i in range(20):
        f.readline()
    
    pr_df = pd.read_csv(f, sep='\t', names=['rsid', 'chromosome', 'position', 'genotype'])
pr_df = pr_df[~pr_df['genotype'].isin(['II', 'DD', 'ID', 'DI'])]
pr_df.to_csv('professor_snp_filtered.txt', sep='\t', header=False, index=False)

610526
596265


In [74]:
!~/Prog/tools/plink_mac/plink --23file ~/Bioinf/Workshop/bioworkshop/H+/professor_snp_filtered.txt --recode vcf --out professor_out

PLINK v1.90b6.16 64-bit (19 Feb 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to professor_out.log.
Options in effect:
  --23file /Users/b2w/Bioinf/Workshop/bioworkshop/H+/professor_snp_filtered.txt
  --out professor_out
  --recode vcf

8192 MB RAM detected; reserving 4096 MB for main workspace.
--23file: professor_out-temporary.bed + professor_out-temporary.bim +
professor_out-temporary.fam written.
864 variants with indel calls present.  '--snps-only no-DI' or
--list-23-indels may be useful here.
Inferred sex: male.
596265 variants loaded from .bim file.
1 person (1 male, 0 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182

In [75]:
!bedtools intersect -a professor_out.vcf -b clinvar.vcf.gz > professor_intersected.vcf

In [1]:
!wc -l professor_snp.txt

  610546 professor_snp.txt


## 1. Who are we?
### a. Origin

In [104]:
df = pd.read_csv('professor_intersected.vcf', sep='\t', 
                 names=['CHR', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE'])
df = df[df['ALT'] != '.']
df = df[df['REF'] != 'N']
print(len(df['ID']))
print(df.head())

2015
    CHR      POS         ID REF ALT QUAL FILTER INFO FORMAT SAMPLE
7     1  2235672  rs2843159   C   T    .      .   PR     GT    0/1
13    1  2494330  rs2234167   A   G    .      .   PR     GT    0/1
16    1  3328358   rs870124   C   T    .      .   PR     GT    0/1
17    1  3328659  rs2493292   C   T    .      .   PR     GT    0/1
18    1  3342530  rs2244013   A   G    .      .   PR     GT    0/1


<img src="prof.ethnicity.png">

### b. Annotation

In [ ]:
!java -Xmx4g -jar ~/Downloads/snpEff_latest_core/snpEff/snpEff.jar -c ~/Downloads/snpEff_latest_core/snpEff/snpEff.config -v "GRCh37.75" ~/Bioinf/Workshop/bioworkshop/H+/professor_intersected.vcf > prof.eff.vcf

## 2. Where are we going?
### a. Fixes

In [ ]:
Entrez.email = 'riserisen@yandex.ru'

In [ ]:
i = 600
for snp_id in df['ID'].values[i, i + 200]:

    handle = Entrez.esearch(db="ClinVar", term=snp_id)
    record = Entrez.read(handle)
    if len(record['IdList']) == 0:
        continue
    record_id = record['IdList'][0]

    summary_handle = Entrez.esummary(db="ClinVar", id=record_id)
    summary = Entrez.read(summary_handle)['DocumentSummarySet']['DocumentSummary'][0]
#     print(summary['allele_freq_set'])
    if summary['clinical_significance'] != 'Benign':
        print(snp_id)
        print(summary['clinical_significance'])
    

In [ ]:
i = 1100

for snp_id in df['ID'].values[i:i+1250]:
    handle = Entrez.esearch(db="SNP", term=snp_id)
    record = Entrez.read(handle)
    i += 1
    if len(record['IdList']) == 0:
        continue
    record_id = record['IdList'][0]

    summary_handle = Entrez.esummary(db="SNP", id=record_id)
    summary = Entrez.read(summary_handle)['DocumentSummarySet']['DocumentSummary'][0]
    if "p" in summary['CLINICAL_SIGNIFICANCE']:
        print(f'--{i}--', snp_id)
        print(summary['CLINICAL_SIGNIFICANCE'])
        print(summary['GENES'])
        allele_freq_list = [float(x['FREQ'][2:5]) for x in summary['GLOBAL_MAFS']]
        allele_freq = sum(allele_freq_list) / len(allele_freq_list)
        print(allele_freq)

|Reference SNP ID|Clinical significance|Gene|MAF|
|---|---|---|---|
|rs448740|pathogenic|FCGR3B|0.33|
|rs3755319|pathogenic|UGT1A10|0.43|
|rs35329108|pathogenic|SLC6A19|0.22|
|rs460897|pathogenic, risk-factor|CFH|?|
|rs1522296|likely-pathogenic|PAH|0.27|
|rs11085825|likely-pathogenic|GCDH|0.2625|

### b. Advantages

|Reference SNP ID|Clinical significance|Gene|MAF|
|---|---|---|---|
|rs1800872|risk-factor,protective|IL10, IL19|0.2825|




--480-- rs5743618
protective
[{'NAME': 'TLR1', 'GENE_ID': '7096'}]
0.375
--494-- rs1229984
protective
[{'NAME': 'ADH1B', 'GENE_ID': '125'}]
0.08600000000000001
--496-- rs1693482
protective
[{'NAME': 'ADH1C', 'GENE_ID': '126'}]
0.36333333333333334

--556-558-- rs16891982
association,protective,benign
[{'NAME': 'SLC45A2', 'GENE_ID': '51151'}]
0.19666666666666666

--1594-- rs1805015
protective
[{'NAME': 'IL4R', 'GENE_ID': '3566'}]
0.13